# Vessel Emissions API Example

## Run this example in [Colab](https://colab.research.google.com/github/SignalOceanSdk/SignalSDK/blob/master/docs/examples/jupyter/VesselEmissionsAPI/VesselEmissionsAPI.ipynb)

#### In this notebook, we will showcase the Vessel Emissions API and supported methods with examples.

## Setup
Install the Signal Ocean SDK:
```
pip install signal-ocean
```
Set your subscription key acquired here: https://apis.signalocean.com/profile

In [ ]:
pip install signal-ocean

In [1]:
signal_ocean_api_key = '' #replace with your subscription key

## Call the Vessel Emissions API
The Vessel Emissions API retrieves information about voyage emissions, vessel emissions and metrics for a single vessel or a vessel class.

In [2]:
from signal_ocean import Connection
from signal_ocean.vessel_emissions import VesselEmissionsAPI

import pandas as pd

connection = Connection(signal_ocean_api_key)
api = VesselEmissionsAPI(connection)

## Examples

### Get Emissions for a voyage

In [3]:
voyage_emissions = api.get_emissions_by_imo_and_voyage_number(imo=9412036, voyage_number=140)
print("Return type is: ", type(voyage_emissions), "\n")

Return type is:  <class 'signal_ocean.vessel_emissions.models.EmissionsEstimation'> 



#### Note that this function returns a custom object. To get a dictionary object, as in the public API response, we can call class method _to_dict()_.

In [4]:
voyage_emissions.to_dict().keys()

dict_keys(['ID', 'IMO', 'VesselName', 'VoyageNumber', 'VesselTypeID', 'VesselClassID', 'StartDate', 'EndDate', 'Deadweight', 'Emissions', 'VesselType', 'VesselClass', 'Quantity', 'TransportWorkInMillionTonneMiles', 'TransportWorkInMillionDwtMiles', 'ContainsEuEmissions'])

### Include consumptions, distances, durations, speed_statistics in the response

In [5]:
voyage_emissions = api.get_emissions_by_imo_and_voyage_number(imo=9412036, voyage_number=140,
                                                              include_consumptions=True,
                                                              include_distances=True,
                                                              include_durations=True,
                                                              include_speed_statistics=True
                                                             )

### Include efficiency metrics in the response

In [6]:
voyage_emissions = api.get_emissions_by_imo_and_voyage_number(imo=9412036, voyage_number=140,
                                                              include_efficiency_metrics=True
                                                             )

### Include EU regulated emissions in the response

In [7]:
voyage_emissions = api.get_emissions_by_imo_and_voyage_number(imo=9412036, voyage_number=140,
                                                              include_eu_emissions=True
                                                             )

### Get Emissions for a vessel

In [8]:
imo = 9412036
vessel_emissions = api.get_emissions_by_imo(imo=imo, include_distances=True, include_eu_emissions=True)

print(f"Return type is: {type(vessel_emissions)}, of {type(vessel_emissions[0])}")

first, last = vessel_emissions[0], vessel_emissions[-1]

print(f"""Return objects consists of {len(vessel_emissions)} voyage emissions estimations for vessel {imo}, for Voyages #{first.voyage_number} to #{last.voyage_number}""" )

Return type is: <class 'list'>, of <class 'signal_ocean.vessel_emissions.models.EmissionsEstimation'>
Return objects consists of 71 voyage emissions estimations for vessel 9412036, for Voyages #106 to #176


#### Note that this function returns a list of custom objects. To get a list of dictionary objects, as in the public API response, we can call method _to_dict()_ in a list comprehension.

In [9]:
emissions_as_dicts = [emissions.to_dict() for emissions in vessel_emissions]

### Get Emissions for a vessel class.
Again, we can get a dictionary, similar to the response of the public API by calling method to_dict().
This dict has two keys, 'NextPageToken' which can be used in the next api call and 'Data', where our emissions are stored as a list of dictionaries. We can apply the pandas DataFrame constructor to that list get our emissions in table format.

In [10]:
vessel_class_emissions = api.get_emissions_by_vessel_class_id(vessel_class_id=86)
vessel_class_emissions_df = pd.json_normalize(vessel_class_emissions.to_dict()['Data'], sep='')
vessel_class_emissions_df.head()

,ID,IMO,VesselName,VoyageNumber,VesselTypeID,VesselClassID,StartDate,EndDate,Deadweight,VesselType,...,EmissionsPortCallSOxInTons,EmissionsPortCallPmInTons,EmissionsStopCO2InTons,EmissionsStopCOInTons,EmissionsStopCh4InTons,EmissionsStopN2OInTons,EmissionsStopNMVOCInTons,EmissionsStopNOxInTons,EmissionsStopSOxInTons,EmissionsStopPmInTons
0,I92EA8CVEDE4C9900,9628300,Hilda Knutsen,183,1,86,2024-08-22T23:56:14,2024-09-10T23:06:37.484000,123166,Tanker,...,0.156405,0.075120,241.371325,0.208546,0.004517,0.011293,0.231885,6.169800,0.147191,0.070695
1,I8F414CVEDE250C00,9388364,Merbabu,181,1,86,2024-07-21T03:49:12,2024-11-12T09:30:00.792000,105746,Tanker,...,0.156405,0.106184,1383.738290,1.216410,0.026348,0.070262,1.352542,40.760697,0.858539,1.321780
2,I906053VED3547E00,9461843,Torm Gloria,58,1,86,2018-10-27T06:15:32,2019-06-03T07:51:23,119456,Tanker,...,3.910120,0.557153,4073.305629,3.622867,0.078474,0.209263,4.028314,114.022214,63.925290,9.108708
3,I8DEDABVEDE4C9900,9301419,Yuri Senkevich,307,1,86,2024-09-02T03:55:44,2024-09-15T12:22:59.041000,100869,Tanker,...,0.156405,0.075120,33.336873,0.029306,0.000635,0.001693,0.032585,0.982002,0.020684,0.031844
4,I95A928VEDE59C800,9808168,Green Aura,64,1,86,2024-08-31T23:45:55,2024-09-17T07:30:23.790000,113000,Tanker,...,0.096189,0.210047,76.735187,0.066300,0.001436,0.003590,0.073719,1.765197,0.001404,0.010114


To get the next page, we repeat the same api call using the token that we got from the previous.

In [11]:
vessel_class_emissions_next_page = api.get_emissions_by_vessel_class_id(
    vessel_class_id=86,
    token=vessel_class_emissions.to_dict()['NextPageToken']
)
pd.json_normalize(vessel_class_emissions_next_page.to_dict()['Data']).head()

,ID,IMO,VesselName,VoyageNumber,VesselTypeID,VesselClassID,StartDate,EndDate,Deadweight,VesselType,...,Emissions.PortCall.SOxInTons,Emissions.PortCall.PmInTons,Emissions.Stop.CO2InTons,Emissions.Stop.COInTons,Emissions.Stop.Ch4InTons,Emissions.Stop.N2OInTons,Emissions.Stop.NMVOCInTons,Emissions.Stop.NOxInTons,Emissions.Stop.SOxInTons,Emissions.Stop.PmInTons
0,I8A6CCAVED61C6800,9071818,Athina I,99,1,86,2020-04-18T07:12:38,2020-06-02T11:59:29,96001,Tanker,...,0.537641,0.827736,24.676837,0.021693,0.000470,0.001253,0.024120,0.726904,0.015311,0.023572
1,I8A6CCAVED643F500,9071818,Athina I,100,1,86,2020-06-02T11:59:29,2020-06-12T07:58:33,96001,Tanker,...,0.156405,0.240796,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,I8A6CCAVED66B8200,9071818,Athina I,101,1,86,2020-06-12T07:58:33,2020-08-03T15:57:16,96001,Tanker,...,0.185731,0.285945,4.296830,0.003777,0.000082,0.000218,0.004200,0.126571,0.002666,0.004104
3,I8A6CCAVED6930F00,9071818,Athina I,102,1,86,2020-08-03T15:57:16,2020-09-04T15:57:13,96001,Tanker,...,0.185731,0.285945,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,I8A6CCAVED6E22900,9071818,Athina I,103,1,86,2020-09-04T15:57:13,2020-10-03T19:54:35,96001,Tanker,...,0.185731,0.285945,2.549097,0.002241,0.000049,0.000129,0.002492,0.075089,0.001582,0.002435


### *Vessel Metrics*

In [12]:
vessel_metrics = api.get_metrics_by_imo(imo=9412036)

print("Return type is: ", type(vessel_metrics), "of", type(vessel_metrics[0]), "\n")

print("Return objects is:\n", )
vessel_metrics[0]

Return type is:  <class 'list'> of <class 'signal_ocean.vessel_emissions.models.VesselMetrics'> 

Return objects is:



VesselMetrics(imo=9412036, year=2018, vessel_type=Tanker, vessel_type_id=1, vessel_class=Aframax, vessel_class_id=86, eexi=Eexi(value=4.205015419201572, unit='g-CO2/ton mile', required=3.288153751950862), eiv=Eiv(value=4.69363738382239, unit='g-CO2/ton mile'), aer=Aer(value=4.370328069517782, unit='g-CO2/dwt mile', poseidon_principles_class='Oil Tanker 80000-119999 DWT', poseidon_principles_alignment_in_percentage=11.204276578060616, poseidon_principles_year_target=3.93), cii=Cii(value=4.247319824890369, unit='g-CO2/capacity mile', rating='C', target=4.264058648014019, target_year=2018))

#### Since our return object is a list, we need to use _to_dict()_ method and list comprehension to get a list of dictionaries. Then, using pd.DataFrame we get our metrics in table format.

In [13]:
pd.json_normalize([metrics.to_dict() for metrics in vessel_metrics], sep='').head()

,IMO,Year,VesselType,VesselTypeID,VesselClass,VesselClassID,EexiValue,EexiUnit,EexiRequired,EivValue,...,AerValue,AerUnit,AerPoseidonPrinciplesClass,AerPoseidonPrinciplesAlignmentInPercentage,AerPoseidonPrinciplesYearTarget,CiiValue,CiiUnit,CiiRating,CiiTarget,CiiTargetYear
0,9412036,2018,Tanker,1,Aframax,86,4.205015,g-CO2/ton mile,3.288154,4.693637,...,4.370328,g-CO2/dwt mile,Oil Tanker 80000-119999 DWT,11.204277,3.93,4.247320,g-CO2/capacity mile,C,4.264059,2018
1,9412036,2019,Tanker,1,Aframax,86,4.205015,g-CO2/ton mile,3.288154,4.693637,...,3.578199,g-CO2/dwt mile,Oil Tanker 80000-119999 DWT,-6.817738,3.84,3.609029,g-CO2/capacity mile,B,4.264059,2019
2,9412036,2020,Tanker,1,Aframax,86,4.205015,g-CO2/ton mile,3.288154,4.693637,...,3.687948,g-CO2/dwt mile,Oil Tanker 80000-119999 DWT,-1.391768,3.74,3.676282,g-CO2/capacity mile,B,4.221418,2020
3,9412036,2021,Tanker,1,Aframax,86,4.205015,g-CO2/ton mile,3.288154,4.693637,...,4.081832,g-CO2/dwt mile,Oil Tanker 80000-119999 DWT,12.138239,3.64,4.065335,g-CO2/capacity mile,C,4.178777,2021
4,9412036,2022,Tanker,1,Aframax,86,4.205015,g-CO2/ton mile,3.288154,4.693637,...,3.732896,g-CO2/dwt mile,Oil Tanker 80000-119999 DWT,5.449053,3.54,3.732896,g-CO2/capacity mile,B,4.136137,2022


### Get Metrics for a vessel for a specific year

In [14]:
vessel_metrics = api.get_metrics_by_imo(imo=9412036, year=2020)

### Get Metrics for a vessel class

In [15]:
vessel_class_metrics = api.get_metrics_by_vessel_class_id(vessel_class_id=86)

### Get Metrics for a vessel class for a specific year

In [16]:
vessel_class_metrics_2020 = api.get_metrics_by_vessel_class_id(vessel_class_id=86, year=2020)